# Данные для отчетов и цели занятия
Данные счетчиков сайтов по умолчанию закрыты. Крайне редко кто-то делает их общедоступными. Для целей нашего курса используются данные моего сайта, посвященного спортивной тематике. Чтобы в материалах и упражнениях мы получали одинаковые ответы. Те же самые функции и приемы будут работать на данных Яндекс.Метрики ваших рабочих и личных сайтов.

Мы научимся выгружать отчеты из Яндекс.Метрики на примере нескольких различных отчетов. Это позволит вам при работе со своими сайтами оперативно создавать отчеты нужных конфигураций, отвечающие задачам бизнеса. Сейчас мы будем использовать различные комбинации измерений и метрик, чтобы понять принципы работы с API.

# Как устроен запрос отчета
Итак, давайте посмотрим как получать отчеты Яндекс.Метрики по API. В этом блоке мы изучим как запрашивать и обрабатывать основные типы отчетов: по основным метрикам, работа с шаблонами отчетов и применение фильтров.

Для получения отчета Метрики необходимо указать нужные параметры в запросе. Перечислим основные из них:

URL запроса https://api-metrika.yandex.ru/stat/v1/data
date1 и date2 - начальная и конечная дата выгрузки
id - номер счетчика, данные которого мы запрашиваем
metrics - список метрик отчета (что будет в столбцах отчета)
dimensions - список измерений отчета (что будет в строках)
oauth_token - авторизационный токен (в текущем модуле мы используем счетчик с открытой статистикой, поэтому ваши запросы будут работать и без токена)
Номер счетчика используемого для тренировки сайта 21075004.

Получим статистику по основным метрикам проекта за неделю 12-18 февраля 2018. Т. е. получим данные как в отчете Посещаемость веб-интерфейса Метрики:

In [1]:
import requests
from yaml import load
from pprint import pprint

In [2]:
with open('./module12_files/yandex_config.yaml', 'r') as f:
    token = load(f)['access_token']

В качестве строк (измерений) отчета берем дату визита, метрик - визиты (ym:s:visits), посетители (ym:s:users), просмотры (ym:s:pageviews) и доля новых пользователей (ym:s:percentNewVisitors):

In [3]:
api_url = 'https://api-metrika.yandex.ru/stat/v1/data'
# Задаем даты выгрузки и номер счетчика:
startDate = '2018-02-12'
endDate = '2018-02-18'
counter = '21075004'

dimensions = ['ym:s:date']
metrics = ['ym:s:visits', 
           'ym:s:users', 
           'ym:s:pageviews', 
           'ym:s:percentNewVisitors']

Измерения и метрики для запроса к API нужно передавать как строку. Преобразуем наш лист метрик и измерений в строку:

In [4]:
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions) #для одного измерения не имеет смысла =)
print(metrics_string, dimensions_string, sep='\n')

ym:s:visits,ym:s:users,ym:s:pageviews,ym:s:percentNewVisitors
ym:s:date


Остается свести все переменные в параметры запроса:

In [5]:
params = {
    'date1': startDate,
    'date2': endDate,
    'id': counter, #счетчик
    'dimensions': dimensions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

# Отправляем запрос к API
Теперь мы посмотрим, как отправлять запрос к API и обрабатывать ответы.

Отправляем запрос и смотрим ответ:

In [6]:
r = requests.get(url=api_url, params=params)
data = r.json()

In [7]:
pprint(data)

{'data': [{'dimensions': [{'name': '2018-02-17'}],
           'metrics': [480.0, 375.0, 921.0, 72.8]},
          {'dimensions': [{'name': '2018-02-16'}],
           'metrics': [403.0, 358.0, 668.0, 73.46368715]},
          {'dimensions': [{'name': '2018-02-18'}],
           'metrics': [329.0, 263.0, 696.0, 64.25855513]},
          {'dimensions': [{'name': '2018-02-13'}],
           'metrics': [320.0, 297.0, 471.0, 80.80808081]},
          {'dimensions': [{'name': '2018-02-12'}],
           'metrics': [310.0, 271.0, 487.0, 78.59778598]},
          {'dimensions': [{'name': '2018-02-14'}],
           'metrics': [308.0, 281.0, 508.0, 79.71530249]},
          {'dimensions': [{'name': '2018-02-15'}],
           'metrics': [308.0, 278.0, 482.0, 75.53956835]}],
 'data_lag': 110,
 'max': [480.0, 375.0, 921.0, 80.80808081],
 'min': [308.0, 263.0, 471.0, 64.25855513],
 'query': {'attribution': 'Last',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2018-02-12

В ответе Метрики есть много вспомогательной информации для отчета. Сейчас нас интересует таблица по ключу 'data'. Пройдемся по всем этапам получения дат и метрик в этом ответе. Таким образом можно постепенно понять структуру любого JSON-ответа:

In [8]:
pprint(data['data'])

[{'dimensions': [{'name': '2018-02-17'}],
  'metrics': [480.0, 375.0, 921.0, 72.8]},
 {'dimensions': [{'name': '2018-02-16'}],
  'metrics': [403.0, 358.0, 668.0, 73.46368715]},
 {'dimensions': [{'name': '2018-02-18'}],
  'metrics': [329.0, 263.0, 696.0, 64.25855513]},
 {'dimensions': [{'name': '2018-02-13'}],
  'metrics': [320.0, 297.0, 471.0, 80.80808081]},
 {'dimensions': [{'name': '2018-02-12'}],
  'metrics': [310.0, 271.0, 487.0, 78.59778598]},
 {'dimensions': [{'name': '2018-02-14'}],
  'metrics': [308.0, 281.0, 508.0, 79.71530249]},
 {'dimensions': [{'name': '2018-02-15'}],
  'metrics': [308.0, 278.0, 482.0, 75.53956835]}]


Теперь уже видно, что дата лежит во вложенном словаре, а значения метрик заключены в листе под ключом 'metrics' (порядок метрик такой же как мы указывали в переменной metrics_string). Выведем значения метрик:

In [9]:
for line in data['data']:
    visit_date = line['dimensions'][0]['name']
#     visits, users, pageviews, percent_new_visitors = line['metrics']
#     print(visit_date, visits, users, pageviews, percent_new_visitors)
    print(visit_date, *line['metrics'])

2018-02-17 480.0 375.0 921.0 72.8
2018-02-16 403.0 358.0 668.0 73.46368715
2018-02-18 329.0 263.0 696.0 64.25855513
2018-02-13 320.0 297.0 471.0 80.80808081
2018-02-12 310.0 271.0 487.0 78.59778598
2018-02-14 308.0 281.0 508.0 79.71530249
2018-02-15 308.0 278.0 482.0 75.53956835


# Упражнение
(1 балл из 1)
Посчитайте среднее значение доли новых посетителей в этом ответе по дням. Т. е. посчитайте отношение суммы новых пользователей за каждый день к сумме всех пользователей за каждый день. Ответ округлите до второго знака после запятой. Формат ответа: 0.79

In [10]:
sum_new_users = 0
sum_all_users = 0
sum_perc = 0

for line in data['data']:
    visit_date = line['dimensions'][0]['name']
    visits, users, pageviews, percent_new_visitors = line['metrics']
    sum_new_users += users * percent_new_visitors
    sum_all_users += users
    sum_perc += percent_new_visitors
print('Mean new users percentage {:.5f}'.format(sum_new_users / sum_all_users * 0.01))
print('Mean new users percentage {:.5f}'.format(sum_perc / len(data['data']) * 0.01))

#Первое и второе предложение условия не совсем согласованы, 
# цифры получаются разные, хотя при округлении - результат 0,75

Mean new users percentage 0.74988
Mean new users percentage 0.75026


# Подсчет уникальных пользователей
Доля новых пользователей в веб-интерфейсе за всю неделю оказалась заметно выше (значение 85.9% в строке "Итого и средние"), чем за каждый день в отдельности (за 18.02 - 64,3%, за 17.02 - 72,8% итд). Какой результат верен - итоговый или по дням? Ведь не может среднее за неделю быть больше, чем значение за каждый день в отдельности. Выберите вариант ответа внизу страницы.

<img src='./module12_files/m8_b4_new_share.JPG'>

 Оба результата верны, т. к. сумма посетителей за 7 дней не равна недельной аудитории. А для новых посетителей сумма за 7 дней равна недельной сумме 

# Шаблоны отчетов
В API Яндекс.Метрики предусмотрен вариант получения преднастроенных отчетов с помощью шаблонов. Это может существенно сократить время написания запроса, т. к. не требует перечисления названий всех необходимых нам метрик. 

Например, отчет по основным метрикам можно получить, используя параметр preset.

Подробное описание отчета: https://tech.yandex.ru/metrika/doc/api2/api_v1/presets/preset_traffic-docpage/

Там же приведены примеры других вариантов шаблонов. В этом шаге будем использовать шаблон 'traffic'.

Настройки запроса остаются с прошлого шага:

# Упражнение
(1 возможный балл)
Сколько различных метрик мы получили в этом отчете?

In [11]:
preset = 'traffic'
params = {
    'date1': startDate,
    'date2': endDate,
    'id': counter, #счетчик
    'preset': preset,
    'oauth_token': token
}
data = requests.get(url=api_url, params=params).json()

In [12]:
len(data['data'][0]['metrics'])

7

# Фильтры в отчетах
На данные отчета можно накладывать условия. Это очень удобно, когда вам необходимо выгрузить набор отчетов с определенным параметром, например узнать метрики по дням только для России. Не всегда эту задачу могут решить готовые шаблоны. 

**Сложное решение** . Можно сделать новую конфигурацию отчета, добавив в список измерений разбивку на страны. И уже после выгрузки данных мы можем отфильтровать таблицу по названию страны.

Очевидно, это не всегда удобно, т. к. вся таблица может не поместиться в один запрос (о том как выгружать многостраничные отчеты мы поговорим в следующих блоках).

**Простое решение**. Можно просто наложить к отчету из прошлого шага фильтр на страну.

Пример фильтра посещений из России:

In [13]:
filters = "ym:s:regionCountryName=='Россия'"
params = {
    'date1': startDate,
    'date2': endDate,
    'id': counter,
    'preset': preset,
    'oauth_token': token,
    'filters': filters
}
data_rus = requests.get(url=api_url, params=params).json()

In [14]:
# data_rus

In [15]:
pprint(data['query']['metrics'])
pprint(data['data'][0]['metrics'])
pprint(data_rus['data'][0]['metrics'])

['ym:s:visits',
 'ym:s:users',
 'ym:s:pageviews',
 'ym:s:percentNewVisitors',
 'ym:s:bounceRate',
 'ym:s:pageDepth',
 'ym:s:avgVisitDurationSeconds']
[2458.0, 1853.0, 4233.0, 85.91473287, 14.15785191, 1.72213181, 129.22375915]
[2282.0, 1695.0, 3994.0, 84.95575221, 14.24189308, 1.75021911, 133.27388256]


In [16]:
metric_names = list(map(lambda s: s.strip('ym:s:'), data['query']['metrics']))
metric_raw = data['data'][0]['metrics']
metric_rus = data_rus['data'][0]['metrics']
metrics_dict = {}

for n, mraw, mrus in zip(metric_names, metric_raw, metric_rus):
#     print(n, mraw, mrus)
    metrics_dict[n] = {'metrics_raw': mraw, 'metrics_rus': mrus}

In [17]:
metrics_dict['visit']['metrics_rus'] / metrics_dict['visit']['metrics_raw']

0.9283970707892596